In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, LSTM, Dense

import pickle

In [6]:
# load the word_to_id mapping from a file
with open('utils/word_to_id_mapping.pkl', 'rb') as f:
    word_to_id = pickle.load(f)

# load the id_to_word mapping from a file
with open('utils/id_to_word_mapping.pkl', 'rb') as f:
    id_to_word = pickle.load(f)
    
# load the tokenized sentences from a file
with open('utils/tokenized_sentences.pkl', 'rb') as f:
    tokenized_sentences = pickle.load(f)

# load the sentences from a file
with open('utils/sentences.pkl', 'rb') as f:
    sentences = pickle.load(f)
    

In [72]:
# Hyperparameters
embedding_dim = 128
hidden_units = 128
max_sentence_length = 40

# Define the model
def build_lstm_model(vocab_size):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sentence_length))
    model.add(LSTM(units=hidden_units))
    model.add(Dense(units=vocab_size, activation='softmax'))
    return model

# Calculate vocabulary size
vocab_size = len(word_to_id)

# Build the model
model = build_lstm_model(vocab_size)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

checkpoint_callback = ModelCheckpoint(filepath='model_checkpoint_{epoch:02d}.h5', save_freq=5000)


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 40, 128)           6400128   
                                                                 
 lstm_5 (LSTM)               (None, 128)               131584    
                                                                 
 dense_5 (Dense)             (None, 50001)             6450129   
                                                                 
Total params: 12,981,841
Trainable params: 12,981,841
Non-trainable params: 0
_________________________________________________________________
None


In [78]:
import keras

class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        sentence = ['dawno', 'dawno', 'temu']  
        seed_context = [word_to_id[word] for word in sentence if word in word_to_id]

        num_words_to_generate = 20  

        sentence_added = 'dawno dawno temu'
        for _ in range(num_words_to_generate):
            padded_context = np.array([seed_context + [word_to_id['<PAD>']] * (max_sentence_length - len(seed_context))])
            predicted_probs = self.model.predict(padded_context, verbose=0)
            predicted_word_id = np.argmax(predicted_probs)
            predicted_word = id_to_word[predicted_word_id]
            sentence_added += " " + predicted_word
            seed_context.append(predicted_word_id)
            seed_context = seed_context[-max_sentence_length:] 
        print()
        print(sentence_added)

In [79]:
def generator(batch_size):
    while True:
        batch_contexts = []
        batch_targets = []
        
        for _ in range(batch_size):
            idx = np.random.randint(len(tokenized_sentences))
            sentence = tokenized_sentences[idx]
            id_sentence = [word_to_id[word] for word in sentence if word in word_to_id]

            split_idx = np.random.randint(0, min(len(id_sentence), max_sentence_length))

            context = id_sentence[:split_idx]
            target = id_sentence[split_idx]

            context = context + [word_to_id['<PAD>']] * (max_sentence_length - len(context))
            
            batch_contexts.append(context)
            batch_targets.append(target)

        yield np.array(batch_contexts), to_categorical(batch_targets, num_classes=len(word_to_id))


batch_size = 32 

data_generator = generator(batch_size)

model.fit(data_generator, steps_per_epoch=len(tokenized_sentences) // batch_size, callbacks=[checkpoint_callback, CustomCallback()], epochs=100)

Epoch 1/100
7276/7276 [==============================] - ETA: 0s - loss: 8.3671 - accuracy: 0.0295
dawno dawno temu się się i i i i i i i i i i i i i i i i i i
7276/7276 [==============================] - 1244s 171ms/step - loss: 8.3671 - accuracy: 0.0295
Epoch 2/100
7276/7276 [==============================] - ETA: 0s - loss: 8.0869 - accuracy: 0.0377
dawno dawno temu nie nie nie nie nie nie nie nie nie nie nie nie nie nie nie nie nie nie nie nie
7276/7276 [==============================] - 1253s 172ms/step - loss: 8.0869 - accuracy: 0.0377
Epoch 3/100
7276/7276 [==============================] - ETA: 0s - loss: 7.9342 - accuracy: 0.0445
dawno dawno temu się i i nie ma się i i nie ma się i i nie ma się i i nie ma
7276/7276 [==============================] - 1241s 171ms/step - loss: 7.9342 - accuracy: 0.0445
Epoch 4/100
7276/7276 [==============================] - ETA: 0s - loss: 7.8857 - accuracy: 0.0500
dawno dawno temu się w i i i nie ma się i i nie ma się i i i i i i i
7276/7276 [=

In [28]:
# load the model
from tensorflow.keras.models import load_model
model = load_model('model_checkpoint_13.h5')


In [29]:
sentence = ['dawno', 'dawno', 'temu']  # Assuming you have tokenized_sentences defined
seed_context = [word_to_id[word] for word in sentence if word in word_to_id]

max_sentence_length = 40

num_words_to_generate = 40  # Adjust as needed

sentence_added = 'dawno dawno temu'
for _ in range(num_words_to_generate):
    padded_context = np.array([seed_context + [word_to_id['<PAD>']] * (max_sentence_length - len(seed_context))])
    predicted_probs = model.predict(padded_context, verbose=0)
    predicted_word_id = np.argmax(predicted_probs)
    predicted_word = id_to_word[predicted_word_id]  # Assuming you have id_to_word mapping
    sentence_added += " " + predicted_word
    seed_context.append(predicted_word_id)
    seed_context = seed_context[-max_sentence_length:]  # Keep context within max_sentence_length
print()
print(sentence_added)


dawno dawno temu nie wiedział co począć z tego nie było w tym porządku że nie było w tym rzeczach i na razie nie było ani słowa ani słowa ani ani ani ani ani oka nie nie nie na to łzach pojmujecie wolnej
